Data source: https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies?resource=download

In [1]:
from pyspark.sql.functions import col, count, when, isnan,round, to_date

StatementMeta(, 408ba8a6-01fd-4f74-a962-db1ea3069778, 3, Finished, Available, Finished)

In [2]:
#start_date = "2025-01-27" #{start_date}

StatementMeta(, 408ba8a6-01fd-4f74-a962-db1ea3069778, 4, Finished, Available, Finished)

In [3]:
df = spark.read.format("csv").option("header","true").load(f"Files/ott/{start_date}/*.csv")
# df now is a Spark DataFrame containing CSV data from "Files/ott/2023-01-07 310512.244212.csv".
#display(df)

StatementMeta(, 408ba8a6-01fd-4f74-a962-db1ea3069778, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2557ec70-42cd-4c96-a86c-789c80f755f1)

In [37]:
#df.printSchema()

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 39, Finished, Available, Finished)

In [38]:
# Check for duplicates
#df.groupBy(df.columns).count().filter(col("count") > 1).show()

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 40, Finished, Available, Finished)

In [39]:
# remove duplicates
df_nodups = df.dropDuplicates()

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 41, Finished, Available, Finished)

In [40]:
# Get count of NULL values for all columns:

display(df_nodups.select(\

        [count(\

            when(isnan(c) | col(c).isNull(), c)\

             ).alias(c) for c in df_nodups.columns]

        ))

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 42, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8a54d6ac-ae19-4f5e-b5eb-a269073839f4)

In [41]:
##----Drop only if entire row is NULL

df_nodups = df_nodups.na.drop(how='all')

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 43, Finished, Available, Finished)

In [42]:
df_nodups.select(\

        [count(\

            when(isnan(c) | col(c).isNull(), c)\

             ).alias(c) for c in df_nodups.columns]

        ).show()

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 44, Finished, Available, Finished)

+---+-----+------------+----------+------+------------+-------+-------+-----+------+-------+-----------------+----------+------+----------------+---------+
| id|title|vote average|vote_count|status|release date|revenue|runtime|adult|budget|imdb_id|original_language|popularity|genres|spoken_languages|AddedDate|
+---+-----+------------+----------+------+------------+-------+-------+-----+------+-------+-----------------+----------+------+----------------+---------+
|  0|    0|           0|         0|     0|           0|      0|      0|    0|     0|      0|                0|         0|     8|              14|        0|
+---+-----+------------+----------+------+------------+-------+-------+-----+------+-------+-----------------+----------+------+----------------+---------+



In [43]:
#Replacing null values in Language
df_nonulls = df_nodups.na.fill(value='No Language',subset="spoken_languages")

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 45, Finished, Available, Finished)

In [44]:
#df_nonulls.filter(col('spoken_languages') == 'No Language').show()

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 46, Finished, Available, Finished)

In [45]:
#df_nonulls.filter(col('genres') == 'Unknown').show()

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 47, Finished, Available, Finished)

In [46]:
##- Creating new column based on IMDB sourcecontainer

df = df_nonulls.withColumn("IMDB_Category",\

           when(col("vote average").between(1,2.9),"Very Low")\

           .when(col("vote average").between(3,4.9) , "Low")\

           .when(col("vote average").between(5,7.9), "Medium")\

           .otherwise("High"))



StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 48, Finished, Available, Finished)

In [47]:
#Create a new column making runtime in mins to Runtime in hours
df_new = df.withColumn('RuntimeInHours',round(df.runtime/60,2))

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 49, Finished, Available, Finished)

In [48]:
df_new=df_new.withColumnRenamed("runtime","RuntimeInMins").withColumnRenamed("vote average","vote_average")\
            .withColumnRenamed("release date","release_date")


StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 50, Finished, Available, Finished)

In [49]:
## -- Runtime category

df_new = df_new.withColumn(
           "Runtime_Category",
           when(col("RuntimeInHours").between(0,1.30), "Short Runtime")
           .when(col("RuntimeInHours").between(1.31,2.15) , "Medium Runtime")
           .otherwise("LongRuntime"))

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 51, Finished, Available, Finished)

In [50]:
#df_new.select('AddedDate').show(truncate=False)

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 52, Finished, Available, Finished)

In [51]:
#Converting String to date datatypes
df_final = df_new.withColumn('AddedDate',to_date('AddedDate','dd/MM/yyyy'))\
        .withColumn('release_date',\
        to_date('release_date','dd/MM/yyyy'))

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 53, Finished, Available, Finished)

In [52]:
#display(df_final)

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 54, Finished, Available, Finished)

In [53]:
#display(df_final)

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 55, Finished, Available, Finished)

In [54]:
#display(df_final)

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 56, Finished, Available, Finished)

In [55]:
# overide the data to the table
df_final.write.mode('overwrite').saveAsTable('move_rating_silver')

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 57, Finished, Available, Finished)

In [56]:
#df1 = spark.sql("SELECT * from LH_Earthquake.move_rating_silver")
#display(df1.head(3))

StatementMeta(, f0df4cd6-e692-4979-8e3c-dc8f8561a758, 58, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 33601389-88d7-4f6c-a342-8f9b0e8e210c)